In [ ]:
!pip install transformers==4.24.0
!pip install simpletransformers==0.63.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, balanced_accuracy_score

from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
import sklearn

In [ ]:
cuda_available = torch.cuda.is_available()

def eval_metrics(actual, pred):
    bal_acc = balanced_accuracy_score(actual,pred)
    f1_sc = f1_score(actual,pred,average="micro")
    return bal_acc, f1_sc


def data_loader(filename):
    df = pd.read_csv(filename)
    df.drop(['Unnamed: 0'],axis=1,inplace=True)


    counts = df['medical_specialty'].value_counts()
    others = [k for k,v in counts.items() if v<100]
    for each_spec in others:
        df.loc[df['medical_specialty']==each_spec,'medical_specialty']=' others'

    counts = df['medical_specialty'].value_counts()

    num_classes = len(df['medical_specialty'].unique())
    class_dict = dict(zip(df['medical_specialty'].unique(),list(range(num_classes))))
    df['medical_specialty'] = df['medical_specialty'].apply(lambda x:class_dict[x])
    df.dropna(inplace=True)
    df['transcription'] = df['keywords']+df['transcription']
    X = df[['transcription']]
    y = df[["medical_specialty"]]


    train_x,test_x,train_y,test_y = train_test_split(X,y, stratify=y,test_size=0.25)

    class_weights = [1]*num_classes

    return train_x, train_y, test_x, test_y, num_classes, class_weights, class_dict

In [ ]:
cuda_available

True

In [7]:
if __name__ == '__main__':
    train_x,train_y,test_x,test_y, num_classes, class_weights, class_dict = data_loader("/content/mtsamples.csv")

    train_df = pd.DataFrame(columns=['text','labels'])
    train_df['text'] = train_x['transcription']
    train_df['labels'] = train_y['medical_specialty']
    print(train_df.head())
    test_df = pd.DataFrame(columns=['text','labels'])
    test_df['text'] = test_x['transcription']
    test_df['labels'] = test_y['medical_specialty']


    learning_rate = 1e-5
    num_of_epochs = 3

    model_args = ClassificationArgs(num_train_epochs=num_of_epochs,learning_rate = learning_rate,  reprocess_input_data= True,save_model_every_epoch=False, overwrite_output_dir= True)

    model = ClassificationModel(
        "roberta",
        "roberta-base",
        num_labels=num_classes,
        weight=class_weights,
        use_cuda = False,
        args=model_args
        )

    model.train_model(train_df)
    model.save_model()
    result, model_outputs, wrong_predictions = model.eval_model(test_df)

    result, output = model.predict(test_df['text'].values.tolist())

    acc, f1 = eval_metrics(test_df['labels'], result)

    print(acc,f1)

                                                   text  labels
4588  consult - history and phy., standard bruce, ni...      12
4235  consult - history and phy., digital palpation,...      12
3512  gastroenterology, gia stapler, laparoscopic ap...      10
381   surgery, removal of venous port, acute lymphoc...       5
3356   CHIEF COMPLAINT: , Nausea, vomiting, diarrhea...       4


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

  0%|          | 0/2923 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
class_dict.keys()

In [ ]:
# Compute accuracy and f1 score
acc, f1 = eval_metrics(test_labels, predicted_classes)

print(f' Accuracy = {acc} \n F1 Score = {f1}')

In [ ]:
# Confusion Matrix
cm = confusion_matrix(test_df['labels'], result)

plt.figure(figsize=(25, 20))
ax = plt.subplot()
sns.set(font_scale=2)
sns.heatmap(cm, annot=True, fmt='g', cmap="Blues", ax=ax);

# labels, title and ticks
ax.set_xlabel('Predicted labels', fontsize=20)
ax.set_ylabel('True labels', fontsize=20)

ax.set_title('Confusion Matrix', fontsize=20)

ax.xaxis.set_ticklabels(class_dict.keys(), fontsize=15, rotation = 90)
ax.yaxis.set_ticklabels(class_dict.keys(), fontsize=15, rotation = 0)

plt.show()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_labels, predicted_classes, target_names=class_dict.keys()))